Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from sklearn.metrics import  mean_absolute_error

from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon

import csv
import networkx as nx
import community.community_louvain as community_louvain
import os

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

# **Set the parameters**

<p>Use option = 0 if you want to do training and validation using 2 of the folds (keeping the 3rd fold for final training and testing).</p>

<p>Use option = 1 if you want to do final training (using 2 folds) and testing (with the 3rd fold).</p>

Be sure to update the name of the files - doing 3 cross-validation in this project is done manually with the prepared data to create comparable results with the rest of the projects.



In [ ]:
option = 1

#model parameters
model_parameter = {
    "batch_size" : 16,
    "architecture" : [250,500,800,268,268,268],
    "dropout" : 0.4,
    "input_dim" : 160,
    "output_dim" : 268,
    "epoch" : 15,
    "learning_rate" : 0.001
}

#seed
random_seed = 42

# path
# path for training and validation (option 0)
path_lr_data = 'lr_split_AandB_training.csv'
path_hr_data = 'hr_split_AandB_training.csv'
path_lr_data_test = 'lr_split_AandB_validation.csv'
path_hr_data_test = 'hr_split_AandB_validation.csv'

# path for final training and testing (option 1)
path_lr_data = 'lr_split_AandB_finaltraining.csv'
path_hr_data = 'hr_split_AandB_finaltraining.csv'
path_lr_data_test = 'lr_clusterC.csv'
path_hr_data_test = 'hr_clusterC.csv'

# path to save the evaluation metrics to - this means that Cluster C will be used for testing
path_eval_matrics = '03-clusterCV-split3.csv'

Reproducibility

In [ ]:
# Set a fixed random seed for reproducibility across multiple libraries
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Check for CUDA (GPU support) and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

Intermediate functions

In [ ]:
class MatrixVectorizer:
    """
    A class for transforming between matrices and vector representations.

    This class provides methods to convert a symmetric matrix into a vector (vectorize)
    and to reconstruct the matrix from its vector form (anti_vectorize), focusing on
    vertical (column-based) traversal and handling of elements.
    """

    def __init__(self):
        """
        Initializes the MatrixVectorizer instance.

        The constructor currently does not perform any actions but is included for
        potential future extensions where initialization parameters might be required.
        """
        pass

    @staticmethod
    def vectorize(matrix, include_diagonal=False):
        """
        Converts a matrix into a vector by vertically extracting elements.

        This method traverses the matrix column by column, collecting elements from the
        upper triangle, and optionally includes the diagonal elements immediately below
        the main diagonal based on the include_diagonal flag.

        Parameters:
        - matrix (numpy.ndarray): The matrix to be vectorized.
        - include_diagonal (bool, optional): Flag to include diagonal elements in the vectorization.
          Defaults to False.

        Returns:
        - numpy.ndarray: The vectorized form of the matrix.
        """
        # Determine the size of the matrix based on its first dimension
        matrix_size = matrix.shape[0]

        # Initialize an empty list to accumulate vector elements
        vector_elements = []

        # Iterate over columns and then rows to collect the relevant elements
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Collect upper triangle elements
                        vector_elements.append(matrix[row, col])
                    elif include_diagonal and row == col + 1:
                        # Optionally include the diagonal elements immediately below the diagonal
                        vector_elements.append(matrix[row, col])

        return np.array(vector_elements)

    @staticmethod
    def anti_vectorize(vector, matrix_size, include_diagonal=False):
        """
        Reconstructs a matrix from its vector form, filling it vertically.

        The method fills the matrix by reflecting vector elements into the upper triangle
        and optionally including the diagonal elements based on the include_diagonal flag.

        Parameters:
        - vector (numpy.ndarray): The vector to be transformed into a matrix.
        - matrix_size (int): The size of the square matrix to be reconstructed.
        - include_diagonal (bool, optional): Flag to include diagonal elements in the reconstruction.
          Defaults to False.

        Returns:
        - numpy.ndarray: The reconstructed square matrix.
        """
        # Initialize a square matrix of zeros with the specified size
        matrix = np.zeros((matrix_size, matrix_size))

        # Index to keep track of the current position in the vector
        vector_idx = 0

        # Fill the matrix by iterating over columns and then rows
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Reflect vector elements into the upper triangle and its mirror in the lower triangle
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1
                    elif include_diagonal and row == col + 1:
                        # Optionally fill the diagonal elements after completing each column
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1

        return matrix

In [ ]:
def load_train_data(path_lr_data,path_hr_data, path_lr_data_test, path_hr_data_test):
    # Load the data

    lr_data = pd.read_csv(path_lr_data)
    hr_data = pd.read_csv(path_hr_data)
    lr_data_test = pd.read_csv(path_lr_data_test)
    hr_data_test = pd.read_csv(path_hr_data_test)

    return lr_data,hr_data, lr_data_test, hr_data_test

In [ ]:
def dataloader(lr_data,hr_data,k=1,shuffle=True):
    """
    Create a torch DataLoader
    """
    tuple_matrices = []
    matrix_size_lr = 160
    matrix_size_hr = 268

    for i in range(lr_data.shape[0]):

        # Preprocess lr_data
        sample_lr = lr_data.iloc[i]
        matrix_lr = MatrixVectorizer.anti_vectorize(sample_lr, matrix_size_lr, include_diagonal=False)

        # Preprocess hr_data
        sample_hr = hr_data.iloc[i]
        matrix_hr = MatrixVectorizer.anti_vectorize(sample_hr, matrix_size_hr, include_diagonal=False)

        # Append lists
        tuple_matrices.append((torch.tensor(matrix_lr), torch.tensor(matrix_hr)))

    # Loading data
    loader = DataLoader(tuple_matrices, batch_size=k, shuffle=shuffle, pin_memory=True)

    return loader

In [ ]:
# Early stop implementation to not overfit model
class EarlyStopper:
    def __init__(self, patience=4):
        self.patience = patience
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
def train(model, data_loader, epochs, learning_rate):
    """
    Create a torch DataLoader
    """
    optimizer = Adam(model.parameters(), lr=learning_rate)
    model.train()
    if option == 0:
        early_stopper = EarlyStopper(patience=4)
    for epoch in range(epochs):
        total_loss = 0
        model.train()
        for X_lr, X_hr in data_loader:
            X_lr, X_hr = X_lr.to(device), X_hr.to(device)
            optimizer.zero_grad()
            outputs = model(X_lr)
            loss = F.l1_loss(outputs.flatten(), X_hr.flatten())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if option == 0 and early_stopper.early_stop(total_loss/len(data_loader)):
            break
        print(f"Epoch {epoch+1}, Loss Train: {total_loss/len(data_loader)}")


In [ ]:
def predict(model,test_data_loader):
    """
    Make prediction using model across test data
    """
    predicted = torch.empty((len(test_data_loader),268,268),device=device,dtype=torch.float64)
    real = torch.empty((len(test_data_loader),268,268),device=device,dtype=torch.float64)
    model.eval()

    for i,(X_lr, X_hr) in enumerate(test_data_loader):
            X_lr, X_hr = X_lr.to(device), X_hr.to(device)
            outputs = model(X_lr)[0]
            predicted[i] = outputs
            real[i] = X_hr[0]

    return predicted,real

In [ ]:
def matrix_list_to_data(matrix_list:np.ndarray)->list:
    """
    convert list of vectorize array to suitable shape
    """
    data = []
    for i in range(len(matrix_list)):
        for j in range(matrix_list.shape[1]):
            data.append([i*matrix_list.shape[1]+j+1]+[matrix_list[i][j]])
    return data

def to_csv(data:list, fold_num:int=0):
    """
    creates a csv file with the given data
    """
    print(f"writing {len(data)} out of rows 4007136 to predictions_fold_{fold_num}.csv")
    with open(f"predictions_fold_{fold_num}.csv", 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["ID","Predicted"])
        writer.writerows(data)

def write(matrix_list,fold):
    """
    Write a csv from list of array
    """
    data = matrix_list_to_data(matrix_list)
    to_csv(data,fold)

In [ ]:
def convert_nvec_to_vec(data):
    """
    Convert non vectorize array to vectorize one
    """
    data_vec = np.empty((data.shape[0],35778))

    for i in range(data.shape[0]):
        data_vec[i] = MatrixVectorizer.vectorize(data[i],include_diagonal=False)
    return data_vec

In [ ]:
def analysis_non_vectorize(predicted,real):
    """
    Compute several metrics over non vectorize array
    """
    # Initialize lists to store MAEs for each centrality measure
    mae_bc = []
    mae_ec = []
    mae_pc = []

    # Iterate over each test sample
    for i in range(predicted.shape[0]):
        # Convert adjacency matrices to NetworkX graphs
        pred_graph = nx.from_numpy_array(predicted[i], edge_attr="weight")
        gt_graph = nx.from_numpy_array(real[i], edge_attr="weight")

        # Compute centrality measures
        pred_bc = nx.betweenness_centrality(pred_graph, weight="weight")
        pred_ec = nx.eigenvector_centrality(pred_graph, weight="weight")
        pred_pc = nx.pagerank(pred_graph, weight="weight")

        gt_bc = nx.betweenness_centrality(gt_graph, weight="weight")
        gt_ec = nx.eigenvector_centrality(gt_graph, weight="weight")
        gt_pc = nx.pagerank(gt_graph, weight="weight")

        # Convert centrality dictionaries to lists
        pred_bc_values = list(pred_bc.values())
        pred_ec_values = list(pred_ec.values())
        pred_pc_values = list(pred_pc.values())

        gt_bc_values = list(gt_bc.values())
        gt_ec_values = list(gt_ec.values())
        gt_pc_values = list(gt_pc.values())

        # Compute MAEs
        mae_bc.append(mean_absolute_error(pred_bc_values, gt_bc_values))
        mae_ec.append(mean_absolute_error(pred_ec_values, gt_ec_values))
        mae_pc.append(mean_absolute_error(pred_pc_values, gt_pc_values))

    # Compute average MAEs
    avg_mae_bc = sum(mae_bc) / len(mae_bc)
    avg_mae_ec = sum(mae_ec) / len(mae_ec)
    avg_mae_pc = sum(mae_pc) / len(mae_pc)

    return avg_mae_bc, avg_mae_ec, avg_mae_pc

def analysis_vectorize(predicted,real):
    """
    Compute several metrics over vectorize array
    """
    # Compute metrics
    mae = mean_absolute_error(predicted.reshape(-1), real.reshape(-1))
    pcc = pearsonr(predicted.reshape(-1), real.reshape(-1))[0]
    js_dis = jensenshannon(predicted.reshape(-1), real.reshape(-1))

    return mae, pcc, js_dis

In [ ]:
def plot_hist(l_mae, l_pcc, l_js_dis, l_avg_mae_bc, l_avg_mae_ec, l_avg_mae_pc):
    """
    Plot histogram of several metrics over several fold of data
    """
    metrics = ["MAE","PCC","JSD","MAE(PC)","MAE(EC)","MAE(BC)"]
    c=['pink','blue','green','yellow','grey',"orange"]
    L = [l_mae, l_pcc, l_js_dis, l_avg_mae_pc, l_avg_mae_ec, l_avg_mae_bc]
    for i in range(len(l_mae)):
        plt.title("Fold " +str(i+1))
        plt.bar(metrics,[L[j][i] for j in range(len(L))],color=c)
        plt.show()

    array = np.array(L)
    average = np.mean(array,1)
    std = np.std(array,1)

    plt.title("Average across fold ")
    plt.bar(metrics,average,color=c)
    plt.errorbar(metrics, average, yerr=std, fmt='none', ecolor='red', capsize=4, capthick=2)
    plt.show()

In [ ]:
def normalization(a):
    """
    Adjacency normalization
    """
    s = ((a**2).sum(-1))**0.5+0.000000001
    ds = torch.diag(s[0]**-1)
    return a@ds

In [ ]:
def calculate_centralities(adj_matrix):
    if adj_matrix.shape[0] != adj_matrix.shape[1]:
        raise ValueError(f"Adjacency matrix is not square: shape={adj_matrix.shape}")
    print(f"Processing adjacency matrix of shape: {adj_matrix.shape}")

    G = nx.from_numpy_array(adj_matrix)
    partition = community_louvain.best_partition(G)

    # Calculate the participation coefficient with the partition
    pc_dict = participation_coefficient(G, partition)

    # Calculate averages of centrality measures
    pr = nx.pagerank(G, alpha=0.9)
    ec = nx.eigenvector_centrality_numpy(G, max_iter=100)
    bc = nx.betweenness_centrality(G, normalized=True, endpoints=False)
    ns = np.array(list(nx.degree_centrality(G).values())) * (len(G.nodes()) - 1)
    acc = nx.average_clustering(G, weight=None)

    # Average participation coefficient
    pc_avg = np.mean(list(pc_dict.values()))

    return {
        'pr': np.mean(list(pr.values())),
        'ec': np.mean(list(ec.values())),
        'bc': np.mean(list(bc.values())),
        'ns': ns,
        'pc': pc_avg,
        'acc': acc
    }

def participation_coefficient(G, partition):
    # Initialize dictionary for participation coefficients
    pc_dict = {}

    # Calculate participation coefficient for each node
    for node in G.nodes():
        node_degree = G.degree(node)
        if node_degree == 0:
            pc_dict[node] = 0.0
        else:
            # Count within-module connections
            within_module_degree = sum(1 for neighbor in G[node] if partition[neighbor] == partition[node])
            # Calculate participation coefficient
            pc_dict[node] = 1 - (within_module_degree / node_degree) ** 2

    return pc_dict

def evaluate_all(true_hr_matrices, predicted_hr_matrices, output_path=path_eval_matrics):
    print(true_hr_matrices.shape)
    print(predicted_hr_matrices.shape)

    num_subjects = true_hr_matrices.shape[0]
    results = []

    for i in range(num_subjects):
        true_matrix = true_hr_matrices[i, :, :]
        pred_matrix = predicted_hr_matrices[i, :, :]

        print(f"Evaluating subject {i+1} with matrix shapes: true={true_matrix.shape}, pred={pred_matrix.shape}")

        if true_matrix.shape != pred_matrix.shape or true_matrix.shape[0] != true_matrix.shape[1]:
            print(f"Error: Matrix shape mismatch or not square for subject {i+1}: true={true_matrix.shape}, pred={pred_matrix.shape}")
            continue

        metrics = {
            'ID': i + 1,
            'MAE': mean_absolute_error(true_matrix.flatten(), pred_matrix.flatten()),
            'PCC': pearsonr(true_matrix.flatten(), pred_matrix.flatten())[0],
            'JSD': jensenshannon(true_matrix.flatten(), pred_matrix.flatten()),
        }

        true_metrics = calculate_centralities(true_matrix)
        pred_metrics = calculate_centralities(pred_matrix)

        for key in ['NS', 'PR', 'EC', 'BC', 'PC', 'ACC']:
            metrics[f'MAE in {key}'] = mean_absolute_error([true_metrics[key.lower()]], [pred_metrics[key.lower()]])

        results.append(metrics)

    df = pd.DataFrame(results)
    if not df.empty:
        # Check if the file exists to decide whether to write headers
        file_exists = os.path.isfile(output_path)

        df.to_csv(output_path, mode='a', header=not file_exists, index=False)
        print(f"Results appended to {output_path}.")
    else:
        print("No data to save.")



Cross validation

In [ ]:
def cross_validation(model_ini, train_function, predict_function, path_lr_data, path_hr_data, path_lr_data_test, path_hr_data_test, model_parameter, seed):
    lr_data,hr_data, lr_data_test, hr_data_test = load_train_data(path_lr_data,path_hr_data, path_lr_data_test, path_hr_data_test)

    # Store the fold results
    fold_results = []

    # save metrics
    l_mae = []
    l_pcc = []
    l_js_dis = []
    l_avg_mae_bc = []
    l_avg_mae_ec = []
    l_avg_mae_pc = []

    #retrieve model parameter
    batch_size = model_parameter["batch_size"]
    architecture = model_parameter["architecture"]
    dropout = model_parameter["dropout"]
    input_dim = model_parameter["input_dim"]
    output_dim = model_parameter["output_dim"]
    epoch = model_parameter["epoch"]
    learning_rate = model_parameter["learning_rate"]


        # train and test data
    train_data_lr = lr_data
    train_data_hr = hr_data
    test_data_lr = lr_data_test
    test_data_hr = hr_data_test

        # train/test data loader
    train_loader = dataloader(train_data_lr,train_data_hr,k=batch_size)
    test_loader = dataloader(test_data_lr,test_data_hr,shuffle=False)

        # new model
    model = model_ini(input_dim,output_dim,dropout,architecture)
    model.to(device)

        # train
    train_function(model, train_loader, epoch, learning_rate)

        # predict
    predicted, real = predict_function(model,test_loader)

        # numpy
    np_predicted = predicted.detach().cpu().numpy()
    np_real = real.cpu().numpy()

        # Evaluate the model on the test set and log the results
    metrics = evaluate_all(
        np_real, np_predicted
    )
    fold_results.append(metrics)

        # analysis non vectorize - to generate plots as cited in report
    avg_mae_bc, avg_mae_ec, avg_mae_pc = analysis_non_vectorize(np_predicted,np_real)

        # vectorize - to generate plots as cited in report
    np_predicted = convert_nvec_to_vec(np_predicted)
    np_real = convert_nvec_to_vec(np_real)

        # write file
    write(np_predicted, 1)

        # analysis vectorize
    mae, pcc, js_dis = analysis_vectorize(np_predicted,np_real)

        # save
    l_mae += [mae]
    l_pcc += [pcc]
    l_js_dis += [js_dis]
    l_avg_mae_bc += [avg_mae_bc]
    l_avg_mae_ec += [avg_mae_ec]
    l_avg_mae_pc += [avg_mae_pc]

    return l_mae, l_pcc, l_js_dis, l_avg_mae_bc, l_avg_mae_ec, l_avg_mae_pc

Layers

In [ ]:
class ETN(nn.Module):
    """
    ETn : Edge to node
    Create a form of node from an adjacency matrix
    """
    def __init__(self, i_s, o_s, activation=None):
        super(ETN, self).__init__()
        self.weight = nn.Parameter(torch.DoubleTensor(i_s,o_s), requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(i_s,o_s), requires_grad=True)
        self.activation = activation
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

    def forward(self,x):
        # apply layer
        f = x@(self.weight) + self.bias
        # activation
        h = self.activation(f) if self.activation else f
        return h


class GAT(nn.Module):
    """
    A basic implementation of the GAT layer.

    This layer applies an attention mechanism in the graph convolution process,
    allowing the model to focus on different parts of the neighborhood
    of each node.
    """
    def __init__(self, in_features, out_features, activation=None):
        super(GAT, self).__init__()
        # Initialize the weights, bias, and attention parameters as
        # trainable parameters
        self.weight = nn.Parameter(torch.DoubleTensor(in_features, out_features), requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_features), requires_grad=True)
        self.phi = nn.Parameter(torch.DoubleTensor(2 * out_features, 1), requires_grad=True)
        self.activation = activation
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

        stdv = 1. / np.sqrt(self.phi.size(1))
        self.phi.data.uniform_(-stdv, stdv)

    def forward(self, input, a):
        """
        Forward pass of the GAT layer.

        Parameters:
        input (Tensor): The input features of the nodes.
        adj (Tensor): The adjacency matrix of the graph.

        Returns:
        Tensor: The output features of the nodes after applying the GAT layer.
        """
        input = input.to(device)
        a = a.to(device)

        #normalize adjacency
        adj = normalization(a).to(device)

        # linear transformation and bias
        ltb = (self.bias.reshape(1,-1)+ input@self.weight)

        # Similarity
        S = ((ltb@self.phi[0:self.bias.size()[-1]]).reshape((-1,ltb.size()[-2],1))
             +(ltb@self.phi[self.bias.size()[-1]:]).reshape((-1,1,ltb.size()[-2])))
        S = F.relu(S)

        # mask
        mask = ((adj + torch.eye(adj.size()[-1], device=device)).bool()).to(device)
        S_masked = torch.where(mask, S, torch.tensor(-1e10, device=device))

        h = torch.nn.functional.softmax(S_masked,-1)@ltb

        return self.activation(h) if self.activation else h

class Scaler(nn.Module):
    def __init__(self, d):
        super(Scaler, self).__init__()
        self.weight = nn.Parameter(torch.DoubleTensor(d,d), requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(d,d), requires_grad=True)
        self.reset_parameters()

    def reset_parameters(self):
        stdv1 = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv1, stdv1)

    def forward(self,x):
        # rescale the adjacency to make it similar to the target
        h= x*(self.weight) + self.bias
        return h

class MD(nn.Module):
    def __init__(self, i_s, o_s, activation1=None, activation2=None, activation=None):
        super(MD, self).__init__()
        self.gat = GAT(o_s,o_s,activation=activation1)
        self.etn = ETN(i_s,o_s,activation=activation2)
        self.scaler = Scaler(o_s)
        self.activation = activation

    def forward(self,a):
        # "node" generation
        x = self.etn(a)

        # use of attention
        xx = self.gat(x,a)

        # compute self product to obtain good dimension and symmetry
        h = xx.resize(xx.size(0),xx.size(2),xx.size(1))@xx

        # rescale
        h = self.scaler(h)

        # activation
        h = self.activation(h) if self.activation else h
        h = (1+h)/2
        return h


Model

In [ ]:
class ADAPT(nn.Module):
    """
    Attentive Deep grAph super resoluTion
    """
    def __init__(self,i,o,p,l_s):
        super(ADAPT, self).__init__()
        # dropout
        self.drop = nn.Dropout(p=p)

        # add input and output
        l_s = [i]+l_s+[o]

        # several layers of MD
        self.layers = nn.ModuleList([])
        for j in range(len(l_s)-1):
            self.layers.append(MD(l_s[j], l_s[j+1], activation1=F.tanh, activation2=F.tanh, activation=F.tanh))

    def forward(self,a):
        # apply every layer
        for i in self.layers:
            a = self.drop(a)
            a = i.forward(a)
        return a

Training

In [ ]:
l_mae, l_pcc, l_js_dis, l_avg_mae_bc, l_avg_mae_ec, l_avg_mae_pc = cross_validation(ADAPT, train, predict, path_lr_data, path_hr_data, path_lr_data_test, path_hr_data_test, model_parameter, random_seed)

In [ ]:
# histogram
plot_hist(l_mae, l_pcc, l_js_dis, l_avg_mae_bc, l_avg_mae_ec, l_avg_mae_pc)